# Generador de Recetas



In [287]:
# import numpy as np
%pylab
import pandas as pd
import networkx as nx
import pygraphviz

import seaborn as sns
import itertools
from collections import OrderedDict
from collections import Counter
from networkx.algorithms import approximation as apxa
from networkx.drawing.nx_agraph import write_dot, graphviz_layout
from sklearn import cluster
import random
import numpy as np
# from networkx.algorithms.community import k_clique_communities
from collections import defaultdict
from matplotlib import colors
import nltk

rcParams.update({'font.size': 23})
rc("text", usetex = True)
rc("font", family = "serif")
rcParams['figure.subplot.hspace'] = 0.3
rcParams['figure.subplot.wspace'] = 0.1
mpl.rcParams['lines.linewidth'] = 4

sns.set(style='whitegrid', palette='muted',rc={'figure.figsize': (8,6)})
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)


Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


/anaconda3/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


### La primera parte consiste en limpiar los datos recibidos. 

El dataset recibido contiene numerosos detalles que pueden causar un problemas ne el código:

    1. Filas con títulos y Categoría vacía: Estos datos son inconsistentes y van a ser eliminados
    2. Columnas comenzando por '_' o finalizando en 'asterisco' y 'condición': contienen datos irrelevantes
    3. Las columnas de ingredientes , pasos, entre otras , repinten información después del caracter '|'




In [288]:
#If columns matches with the ending of x, eliminates the repeated patterns in x. 
#    Returns df. with a new column having the cleaned data 
def EliminateRepeated(df, x, column):
    if x.endswith(column) :
            #Separate strings by '|'.
            vecs = df[x].str.split(pat = "|",expand=True)  #vecs is the number of divisions of the df[x]
            num = x.split('_') ; num = num[1]  #get number of column
            df[column.upper()+'_'+ num] = vecs[0]  #create new column with the ingredient
            #Add another ingredient to the columns if there is a diferent value in one of them 
            if 1 in vecs:
                condition = ((vecs[0].notnull()) 
                            & (vecs[1].notnull())
                            & (vecs[0] != vecs[1]) )
                #if the new column provides different results, add the next column with a comma between the values            
                df.loc[condition, column.upper()+'_'+ num ] = (vecs.loc[condition, 0]  
                                                              + ','
                                                              + vecs.loc[condition, 1]) 
            del df[x] #finish eliminating df[x], now that the information have been saved in df[x.upper()+'_'+ num] 
    return df      


def cleanData(df): 
    df =  df.loc[df.Categories == 'Recetas']
    df = df[df.Title.notnull()]
    
    del df['Content']
    del df['Excerpt']
    del df['URL']
    
    for x in df.columns:
        #eliminate vacum columns
        if df[x].count() < 1:
            del df[x]
        #eliminate _ingredient columns. They have useless information    
#         elif x.startswith('_ingredient'):
        elif x.startswith('_'):
            del df[x]
            
        #eliminate aterisco columns. They have useless information     
#         elif x.endswith('asterisco'):
#             del df[x]
#         elif x.endswith('condicion'):
#             del df[x]

    for x in df.columns:
        #Eliminate repeated terms in 'ingredientes', 'cantidades' and 'pasos'
        EliminateRepeated(df, x, 'ingrediente')
        EliminateRepeated(df, x, 'cantidad')
        EliminateRepeated(df, x, 'paso')
    return df       

#Fills null values with  empty strings
#FIND POSSIBLE MISLEADING VALUES ON THE DATA SET AND REPLACE THEM BY NOTHING
def replaceValues(df, x):
    df[x] = df[x].fillna(value = '')
    df[x] = df[x].str.replace('<strong>' , '' )
    df[x] = df[x].str.replace('</strong>' , ':' )
    df[x] = df[x].str.replace('</b>' , ':' )
    df[x] = df[x].str.replace('®' , '' )
    df[x] = df[x].str.replace('<br>' , ':' )
    df[x] = df[x].str.replace('<sup>®</sup>' , '' )
    df[x] = df[x].str.replace('<sup></sup>' , '' )
    df[x] = df[x].str.replace('<sup>®</sup>' , '' )
    df[x] = df[x].str.replace('®' , '' )
    df[x] = df[x].str.replace('<b>' , '' )
    df[x] = df[x].str.replace('<i>' , '' )
    df[x] = df[x].str.replace('</i>' , '' )
    df[x] = df[x].str.replace(')' , '' )
    df[x] = df[x].str.replace('(' , '' )
    df[x] = df[x].str.replace('?' , '' )
    df[x] = df[x].str.replace('¿' , '' )
    df[x] = df[x].str.replace('¡' , '' )
    df[x] = df[x].str.replace('!' , '' )
    df[x] = df[x].str.replace('/' , '' )
    return df[x]
df = pd.read_csv('Recetas.csv', delimiter = ';')
df[{'Title','Categories'}].loc[df.Categories == 'Recetas'].count()
df = cleanData(df)

Ya que tenemos que generar las recetas de acuerdo al tipo de plato (e.g. Postre, Bebida, Plato Fuerte, entrada , etc), es necesario generar una nueva columna llamada 'Tipo'. Para determinar la información en esta vamos a usar la columna 'Tags' la cual contiene palabras clave como 'postre,cena , bebida, etc' que nos permitira asociar rápidamente la receta con el tipo de plato. 

In [289]:
def definirTipo(df):
    df['Tags']  = df.Tags.str.lower()
    #Definir tipo de acuerdo a la aparicion de palabras claves en los TAGS
    df['Tipo'] = ''
    
    df.loc[df.Tags.str.contains(pat = 'desayuno'), 'Tipo' ] = 'Desayuno'
    
    df.loc[ df.Tags.str.contains(pat = 'postre'), 'Tipo' ] = 'Postre'
    df.loc[ df.Tags.str.contains(pat = 'dulce'), 'Tipo' ] = 'Postre'
    df.loc[ df.Tags.str.contains(pat = 'helado'), 'Tipo' ] = 'Postre'
    df.loc[ df.Tags.str.contains(pat = 'torta'), 'Tipo' ] = 'Postre'
    df.loc[ df.Tags.str.contains(pat = 'cheesecake'), 'Tipo' ] = 'Postre'
    df.loc[ df.Tags.str.contains(pat = 'glasé'), 'Tipo' ] = 'Postre'
    df.loc[ df.Tags.str.contains(pat = 'chocolate'), 'Tipo' ] = 'Postre'


    df.loc[ df.Tags.str.contains(pat = 'cena'), 'Tipo' ] = 'PlatoFuerte'
    df.loc[df.Tags.str.contains(pat = 'almuerzo'), 'Tipo' ] = 'PlatoFuerte'
    df.loc[df.Tags.str.contains(pat = 'tacos'), 'Tipo' ] = 'PlatoFuerte'
    

    df.loc[df.Tags.str.contains(pat = 'aperitivos'), 'Tipo' ] = 'Entrada'
    df.loc[df.Tags.str.contains(pat = 'acompañamiento'), 'Tipo' ] = 'Entrada'
    df.loc[df.Tags.str.contains(pat = 'tostada'), 'Tipo' ] = 'Entrada'

    df.loc[df.Tags.str.contains(pat = 'bebida'), 'Tipo' ] = 'Bebida'
    df.loc[df.Tags.str.contains(pat = 'cóctel'), 'Tipo' ] = 'Bebida'
   

    #If empty , it is likely to be an entry
    df.loc[df.Tipo == '', 'Tipo' ] = 'Entrada' 
    return df
df = definirTipo(df)
print(df['Tipo'].value_counts())
df[['Title','Tipo','Tags']].sample(7)

Postre         30
Entrada        24
Bebida         23
PlatoFuerte    17
Desayuno        2
Name: Tipo, dtype: int64


,Title,Tipo,Tags
82,Crujiente de piña,Entrada,avena|azúcar|cocina microondas|horno microonda...
77,Avena rápida con frutas tropicales,Desayuno,almendras|avena|chia|desayuno|fruta tropical|f...
89,Palitos de queso colombianos con salsa de piña,PlatoFuerte,almuerzo|bocadillo|botana|cena|colombia|comida...
101,Arroz con Leche Tropical,Postre,almendras|arroz|arroz con leche|canela|leche c...
98,Flan de Piña,Postre,almidón|ecuador|flan|latinoamérica|leche|piña|...
90,Papaya Tonic con piña,Bebida,agave|bebida|bebida no alcohólica|brindis|cúrc...
85,Crepas brasileñas de tapioca con piña,Postre,brasil|brasileñas|coco|crepa|crepas|crepe|desa...


Gracias a una limpieza efectiva del archivo, hemos encontrado que los datos más relevantes se encuentran en las columnas 
1. Title
2. Tipo
3. Tags
4. Ingredientes
5. Pasos
6. Cantidad


In [290]:
# Clean the cvs_file 
def cleanCVS(cvs_file):
    df = pd.read_csv(cvs_file, delimiter = ';')
    df[{'Title','Categories'}].loc[df.Categories == 'Recetas'].count()
    #Apply previous methods
    df = cleanData(df)
    df = definirTipo(df)
    
    s = ['Title','Tipo', 'Tags']
    df['Tags'] = df['Tags'].str.replace('|' , ' , ' )

    # x = 'INGREDIENTE_ingrediente'
    # df[x] = replaceValues(df, x)
    # df['Total_Ingredientes'] = df['INGREDIENTE_ingrediente']
    # del df['INGREDIENTE_ingrediente']
    
    del df['PASO_nombre']
    del df['PASO_descripcion']
    del df['CANTIDAD_cantidad']
    del df['INGREDIENTE_ingrediente']


    for x in df.columns:
        if x.startswith('INGREDIENTE') :
            df[x] = replaceValues(df, x)
            s.append(x)


    #         df['Total_Ingredientes']  = df['Total_Ingredientes'] + ' , ' + df[x]
    #         del df[x]
    # df['Total_Ingredientes'] = df['Total_Ingredientes'].str.replace(',,' , '' )


#     df['PASOS'] = ""

    for x in df.columns:
         if x.startswith('PASO') :
            df[x] = replaceValues(df, x)
            s.append(x)
         
    #             df['PASOS'] = df['PASOS'] + '.' + df[x]
#   s.append('PASOS')        

    #         s.append(x)
    # new = df['PASOS'].str.split(pat = ".",expand=True)          

    for x in df.columns:
         if x.startswith('CANTIDAD') :
            df[x] = replaceValues(df, x)
            s.append(x)

    df[s].T
    df[s].to_csv('CleanedData.csv')
    return df[s]
df= cleanCVS('Recetas.csv')

In [291]:
display_all(df.describe(include='all').T)

,count,unique,top,freq
Title,96,95,Torta invertida de piña y queso crema,2
Tipo,96,5,Postre,30
Tags,96,96,"almuerzo , brunch , cena , cerdo , chancho , c...",1
INGREDIENTE_0,96,71,,21
INGREDIENTE_1,96,64,,22
INGREDIENTE_2,96,68,,23
INGREDIENTE_3,96,62,,26
INGREDIENTE_4,96,48,,45
INGREDIENTE_5,96,42,,51
INGREDIENTE_6,96,36,,58


# Análisis
En el archivo 'CleanedData.csv' se encuentran las  recetas en una forma mucho más fácil de leer. Analizando ester archivo encontramos los siguientes hallazgos. 
    1. Los pasos a seguir se describen en las columnas 
    2. 

In [292]:
def joinData(df, column):
    df[column + 'S'] = ""
    for x in df.columns:
        if x.startswith(column) :
            sep = '.' if column == 'PASO' else '|'
            df[column + 'S'] = df[column + 'S'] + sep + df[x]
            # s.append('PASOS')        
        #     s.append(x)
    joinedData = list(df[column+ 'S'].str.split(pat = sep))
    
                
    #The data is doubled due to the way the columns are added.
    #These lines will break this discrepancy, hence reducing to the half the lenght of the data
    halfData = []
    for j in joinedData:
        numpas = len(j)
        if(j[: int(numpas/2)] == j[int(numpas/2) :]):
#             print(column, True)
            halfData.append(j[: int(numpas/2)])  
            
    for j in halfData:
            while '' in j:
                j.remove('')
    return df , halfData

def GetListas(df):
    df , pasos = joinData(df, 'PASO')
    df , ingredientes = joinData(df, 'INGREDIENTE')
    df , cantidades = joinData(df, 'CANTIDAD')

    Instrucciones = []  
    Lista_Instrucciones = []

    for p in pasos:
        Inst_p = []
        for x in p:
            x = x.lstrip()
            string = x.split(' ')
            st = string[0].replace(':' , '' )
            Instrucciones.append(st)

            if (len(string)>1):
                if string[1].strip() == 'y':
                    st = st + ' y ' + string[2].replace(':' , '' )
            Inst_p.append(st)
        Lista_Instrucciones.append(Inst_p)
    df['Lista_Instrucciones'] = Lista_Instrucciones 
    return df, pasos , ingredientes, cantidades, Lista_Instrucciones

df, pasos , ingredientes, cantidades, Lista_Instrucciones = GetListas(df)


def printReceta(df , i , pasos , ingredientes, cantidades, Lista_Instruccionces):
    inst = Lista_Instrucciones[i]
    ing = ingredientes[i]
    cant = cantidades[i]
    pas = pasos[i]
    R = array(df['Title'])
    T = array(df['Tipo'])
    print('Receta: ', R[i])
    print('Tipo: ', T[i])

    print('NumIngredientes:', len(ing)
          ,'NumCantidades' ,len(cant)
          ,'NumPasos',len(pas), '\n')
    print('Lista de Instrucciones')
    print(inst)
    print('\n')
    print('Pasos:')
    print(pas)
    print('\n')
    print('Ingredientes -> Cantidades')
    for x in range(len(cant)):
        print(ing[x],'->', cant[x] )
printReceta(df , 10, pasos , ingredientes, cantidades, Lista_Instrucciones )        

Receta:  Julepe de menta con ron y piña
Tipo:  Bebida
NumIngredientes: 5 NumCantidades 4 NumPasos 3 

Lista de Instrucciones
['Frota', 'Llena', 'Agrega']


Pasos:
['Frota: la ramita de menta por el interior del vaso', 'Llena: la mitad del vaso con los cubos molidos de jugo', ' Agrega jugo de piña, azúcar impalpable y ron']


Ingredientes -> Cantidades
Ramita de menta fresca -> 1
Cubos de hielo o hielo molido de jugo de piña DOLE: -> 1 taza
Jugo de piña y naranja o piña DOLE: -> 2 cdas.
Azúcar impalpable -> 1 ½ oz


In [310]:
delated = [    ''
               ,'de'
               ,'y'
               ,'la'
               ,'el'
               ,'en'
               ,'una'
               ,'con'
               ,'los'
               ,'un'
               ,'hasta'
               ,'para'
               ,'a'
               ,'que'
               ,'o'
               ,'las'
               ,'los'
               ,'por'
               ,'al'
               ,'del'
               ,'esté'
               ,'sobre'
               ,'1'
               ,'2'
               ,'3'
               ,'se'
               ,'durante'
               ,'sin'
               ,'coloca'
               ,'minutos']

def plotGraph(i, df):
    df, pasos ,  ingredientes, cantidades, Lista_Instrucciones = GetListas(df)
    inst = Lista_Instrucciones[i]
    ing = ingredientes[i]
    cant = cantidades[i]
    pas = pasos[i]
    R = array(df['Title'])
    T = array(df['Tipo'])
    
    
    G = nx.OrderedDiGraph()
    elist = []
    cols = ['b', 'r']
    G.add_node(T[i])
    name = repr(0) + '.' + inst[0]
    G.add_node(name)
    elist.append((T[i],name))

    # G.add_nodes_from(array(Word_labels))
    for p in range(len(inst)-1):
        cols.append('r')
        name = repr(p) + '.' + inst[p]
        name1 = repr(p+1) + '.'+ inst[p+1]
        G.add_node(name1)
        elist.append((name,name1))
    k = 0
    for j in range(len(ing)):
        words = ing[j].split(' ')
        words = [x.lower().strip() for x in words] #strip each word
        words = list(filter(lambda a: a not in delated, words)) # clean misleading words

        for w in words:
            for p in range(len(pas)):
                if (w in pas[p]) & (w != ''):

                    ingre = repr(p) +')' +ing[j][:13]
                    if ingre not in G.nodes():
                        cols.append('g')

                    canti = repr(p)+','+repr(j) +')' +cant[j]
                    if canti not in G.nodes():
                                cols.append('y')

                    name = repr(p) + '.' + inst[p]
    
                    G.add_node(ingre)
                    G.add_node(canti)
                    elist.append((name, ingre))
                    elist.append((ingre, canti ))
#     n = len(G.nodes())
#     while len(cols) <n:
#         cols.append('y')

    G.add_edges_from(elist)

    # write_dot(G,'test.dot')
    pos = graphviz_layout(G)
    nx.draw_networkx( G, pos
                     , node_color = cols
                     , with_labels = True)
    title(R[i])
    
# plotGraph(19, df)    
# plotGraph(49, df) 
# plotGraph(29, df)
plotGraph(4, df)

# Generación de nuevas recetas

En el último gráfico vimos como cada receta puede expresarse como un grafo(árbol) que tiene 4 niveles de acuerdo con el color. Estos eran:

    1. Tipo -> azul: Tipo de comida
    2. Pasos -> rojo: Pasos de la receta
    3. Ingredientes -> verde : Ingredientes asociados a cada paso
    4. Cantidad -> amarillo : Cantidad asociada a cada ingrediente. 
    
Si nuestro propósito es generar nuevas recetas, es posible hacerlo generando un grafo(árbol) aleatorio ordenado de acuerdo a estas mismas condiciones. 

Para realizar esto, primero vamos a definir los métodos:
    
    - Probabilities : Selecciona las probabilidades del siguiente paso de acuerdo a la cantidad de datos excistentes
    - pickStep : Usando las probabilidades anteriores, elige aleatoriamente el siguiente paso
    - pickIngredients: Elige aleatoriamente los ingredientes con sus respectiva cantidades para el paso actual
    - firstStep : Carga todas las funciones anterios y retorna el primer paso
    - NextStep: Lee todos los datos existentes y calcula el siguiente paso usando las funciones anteriores.

In [405]:
Type = 'Postre'

#Computes the vector of probabilities according to the number of elements in an array
def Probabilities(vec):
    c = Counter(vec)
    h = c.most_common()
    h = array(h)  
    probs = h[:,1].astype(float)
    probs = probs/sum(probs)
    return probs , h # Where h is the counter of vec
    
def pickStep(RealVec,vec):
    if len(vec) == 0:
        return nan
    probs , h = Probabilities(vec)
    Probs = [0] ; cum = 0
    for p in probs:
        Probs.append(cum + p)
        cum = cum + p    
    r = random_sample([2])
    step = h[(r[0] < Probs[1 :]) & (r[0] > Probs[:-1]),0]
    for x in RealVec:
        if x.startswith(step[0]):
            return x 

#Randomly picks a list of possiblel ingredients
#if there is a possible  ingredient with DOLE it always picks it
#Inputs: Lista --- List of possible ingredients
#        Cantidades ----- List of quantities
def pickIngredients(Lista, Cantidades):
    lista = []
    cantidad = []
    if len(Lista) == 0:
        return lista
    else:
        probs , count = Probabilities(Lista)
        for x in range(len(probs)):
            if 'dole' in count[x,0].lower():
                lista.append(count[x,0])
                l = 0 
                found = False
                while (l < len(Lista)-1) & (found == False):
                    l = l +1
                    if count[x,0] == Lista[l]:
                        found = True
                        cantidad.append(Cantidades[l])
                    
            else:
                r = random_sample([2])
                if probs[x]*4 > r[0]:
                    lista.append(count[x,0])
                    l = 0 
                    found = False
                    while (l < len(Lista)-1) & (found == False):
                        l = l + 1 
                        if count[x,0] == Lista[l]:
                            found = True
                            cantidad.append(Cantidades[l])
        return lista , cantidad

    

# Nmax = 6
cut  = 6
def FirstStep(df , tipo):
    #Obtain columns: pasos, ingredientes, Lista_instrucciones
    df, pasos, ingredientes, cantidades, Lista_Instrucciones = GetListas(df)
    pas = array(pasos) ; ing = array(ingredientes) 
    cants = array(cantidades) ; instr = array(Lista_Instrucciones)
    PrimerPaso = [] ; FirstCompare =[] 
    tipo = array(df.Tipo == Type)
    
    #Filte by type of food
    inst_tipo = instr[tipo]
    pas_tipo = pas[tipo]
    ing_tipo = ing[tipo]
    cant_tipo = cants[tipo]
    
    for l in instr:
        PrimerPaso.append(l[0])
        FirstCompare.append(l[0][:cut])
    firstStep = pickStep(PrimerPaso, FirstCompare)
    return firstStep , inst_tipo , pas_tipo , ing_tipo ,cant_tipo


firstStep , inst_tipo , pas_tipo , ing_tipo ,cant_tipo = FirstStep(df , Type)

print('FIRST: ',firstStep)

def NextStep(OldStep):
    Lista = []
    NextInstr = []
    NextInstrSmall =  []
    Cantidades = []
    for l in range(len(inst_tipo)):
        insl = inst_tipo[l]
        for j in range(len(insl)):
            ins = insl[j]
            pasl = pas_tipo[l]
            paso = pasl[j]
            ingreds = ing_tipo[j]
            cantis = cant_tipo[j]
            if(ins.startswith(OldStep[:cut])):
                if (len(insl) > j+1) :
                    if (insl[j+1] not in ['de','Si'] ) :
                        NextInstr.append(insl[j+1])
                        NextInstrSmall.append(insl[j+1][:cut])
    #             print('Instrucción:' , ins, paso)

                if len(ingreds) > 0: 
    #                 print('Ingredientes: ', ingreds)
                    for im in range(len(ingreds)):
                        words = ingreds[im].split(' ')
                        #strip each word
                        words = [x.lower().strip() for x in words] 
                        # clean misleading words
                        words = list(filter(lambda a: a not in delated, words)) 
                        #Encentra los ingredientes posibles para cada paso.
                        for w in words:
                            if(w in paso) & (w != ''):
                                Lista.append(ingreds[im])
                                Cantidades.append(cantis[im])
                                
                                
    return (pickStep(NextInstr,NextInstrSmall)
            , pickIngredients(Lista, Cantidades) )
NextStep(firstStep)



# oldStep , ListaIngredientes = NextStep(f)
# print('Lista:' , Counter(Lista) ) 
# # probs , count = Probabilities(Lista)
# # print(probs,count)
# print('Lista de Ingredientes:' , pickIngredients(Lista))
# print('Next', NextInstr , NextInstrSmall)
# print('Pick ', pickStep(NextInstr,NextInstrSmall))


#                             print('Words: ', w)
# #             
#                 print(ing_tipo[j])



#             words = [x.lower().strip() for x in words] #strip each word
#             words = list(filter(lambda a: a not in delated, words)) # clean misleading words
            
#             for w in words:
#                 if (w in pas[j]) & (w != ''):
#                     print(w,pas[j])
# instr[tipo]
# df2 , ingredientes = joinData(df_tipo, 'INGREDIENTE')
# df2 , cantidades = joinData(df_tipo, 'CANTIDAD')

FIRST:  Coloca


('Procesa',
 (['Rodajas de piña DOLE, sin el jugo',
   'DOLE Pineapple Chunks: piña en trozos, sin el jugo',
   'Jugo DOLE Pineapple: piña',
   'DOLE Mandarin Oranges: mandarinas, sin el jugo',
   'DOLE Jarred Peaches: duraznos envasados, sin el jugo',
   'Azúcar'],
  ['1 lata 20 oz',
   '1 lata 20 oz',
   '14 taza',
   '1 lata 11 o 15 oz',
   '1 taza',
   '1 taza 8 oz']))

In [413]:
Type = 'Postre'
def GenerarReceta(Type , df):

    MaxIt = 10 #Maximum iterations
    Dole = False
    it = 0
    while (not Dole) &  (it < MaxIt) :
        it = it +1
        firstStep , inst_tipo , pas_tipo , ing_tipo ,cant_tipo = FirstStep(df , Type)

        #Initialize Graph
        G = nx.OrderedDiGraph()
        elist = []
        cols = ['b', 'r']
        G.add_node(Type)
        name = repr(0) + '.' + firstStep
        G.add_node(name)
        elist.append((Type,name))


        oldStep = firstStep 

        Nmax = 7
        N = 0
        End = False 

        while (N < Nmax) & (not End):
            nextStep, Ingredients = NextStep(oldStep)

            cols.append('r')
            name = repr(N) + '.' + oldStep


            if nextStep is not nan:
                name1 = repr(N+1) + '.'+ nextStep
                G.add_node(name1)
                elist.append((name,name1))

            if len(Ingredients) > 0: 
                    ingreds = Ingredients[0]
                    cants = Ingredients[1]
                    for j in range(len(cants)):
                        if 'dole' in ingreds[j].lower():
                            Dole = True
                        ingre = repr(N) +')' +ingreds[j][:13]
                        if ingre not in G.nodes():
                            cols.append('g')

                        canti = repr(N)+','+repr(j) +')' +cants[j]
                        if canti not in G.nodes():
                            cols.append('y')

                        G.add_node(ingre)
                        G.add_node(canti)
                        elist.append((name, ingre))
                        elist.append((ingre, canti ))

            #In case the chain stops
            if nextStep is nan:
                End =  True

            #Prepare the next stage
            print(oldStep)
            oldStep = nextStep
            N = N +1

    G.add_edges_from(elist)


    pos = graphviz_layout(G)
    nx.draw_networkx( G, pos
                    , node_color = cols
                    , with_labels = True)


In [409]:
Type = 'Bebida'
GenerarReceta(Type , df)

Coloca
Retíralos
Coloca
Retíralos
Coloca
Deja
Corta


In [410]:
Type = 'Entrada'
GenerarReceta(Type , df)

Vierte
Colócalos
Mezcla
Dejar
Verter
Mezclar
Bate
Bate
Bate


In [412]:
Type = 'PlatoFuerte'
GenerarReceta(Type , df)

Mezcla
Bate
Agrega
Divide
Divide
Tapa
Refrigera


In [ ]:
# UltimoPaso = [] ; CompareU = []
# for l in Lista_Instrucciones:
#     UltimoPaso.append(l[-1])
#     CompareU.append(l[-1][:6])
# c = Counter(CompareU)
# h = c.most_common()
# h = array(h) ; 

# df_tipo['Mezcla' in ]

In [185]:
Lista


print(Lista)
pickIngredients(Lista)

['Comino molido']


['Comino molido']

In [31]:
# ingr_tipo = ingredientes[]
tipo = array(df.Tipo == 'PlatoFuerte')
len(tipo)
len(ingredientes)
ingr = array(ingredientes)
ingr[tipo]

array([list(['DOLE Pineapple Slices: piña en rebanadas, sin el jugo', 'De pollo deshuesadas y sin piel', 'Comino molido', 'Orégano seco desmenuzado', 'Clavo molido', 'Aceite de oliva', 'DOLE Pineapple Juice: jugo de piña', 'Jugo de limón']),
       list([]), list([]),
       list(['Hojas frescas de perejil italiano', 'Aceite de oliva', 'Vinagre de vino rojo', 'Hojas de cilantro fresco']),
       list([]), list([]),
       list(['Mayonesa liviana', 'Jugo de limón o lima', 'Polvo de chile chipotle dividido', 'Lechuga romana DOLE finamente picada']),
       list(['Salchicha huachana', 'Piñas trituradas DOLE Crushed Pineapple, escurridas', 'Sal y pimienta, al gusto', 'Huevos batidos']),
       list(['Piñas trituradas DOLE Crushed Pineapple, escurridas conserva el jugo', 'Harina común, más la cantidad necesaria para amasar', 'Polvo de hornear', 'Azúcar', 'Sal', 'Mantequilla sin sal, a temperatura ambiente', 'Aceite vegetal']),
       list([]),
       list(['Piñas trituradas DOLE Crushed Pin

In [ ]:

# df['PASOS'] = ""
# for x in df.columns:
#     if x.startswith('PASO') :
#         df['PASOS'] = df['PASOS'] + '.' + df[x]
#         # s.append('PASOS')        
#     #     s.append(x)
# pasos = df['PASOS'].str.split(pat = ".")
# pasos = list(pasos)

# df['INGREDIENTES'] = ""
# for x in df.columns:
#     if x.startswith('INGREDIENTE') :
#         df['INGREDIENTES'] = df['INGREDIENTE'] + '.' + df[x]
# ingredientes = df['INGREDIENTE'].str.split(pat = ".")
# ingredientes = list(ingredientes)

# df['CANTIDADES'] = ''
# for x in df.columns:
#     if x.startswith('CANTIDAD') :
#         df['CANTIDADES'] = df['INGREDIENTE'] + '.' + df[x]
# ingredientes = df['INGREDIENTE'].str.split(pat = ".")
# ingredientes = list(ingredientes)


# for p in pasos:
#      while '' in p:
#         p.remove('')



In [ ]:
# Words = list(itertools.chain.from_iterable(Recetas))
Word_labels = list(OrderedDict.fromkeys(Instrucciones))
Word_labels
c = Counter(Instrucciones)
h = c.most_common(200)
array(h)

In [ ]:
pasos[2]

In [ ]:

df = DefinirTipo(df)
df[['Tipo','Tags']]


In [ ]:
r(12)

In [ ]:
data

In [ ]:
data.head()

In [ ]:
[]

In [ ]:
def filterWords(data):
    words = array(data['Title'])
    Recetas = []
    # These words are going to be eliminated from the list
    delated = [ ''
               ,'de'
               ,'y'
               ,'la'
               ,'el'
               ,'en'
               ,'una'
               ,'con'
               ,'los'
               ,'un'
               ,'hasta'
               ,'para'
               ,'a'
               ,'que'
               ,'o'
               ,'las'
               ,'los'
               ,'por'
               ,'al'
               ,'del'
               ,'esté'
               ,'sobre'
               ,'1'
               ,'2'
               ,'3'
               ,'se'
               ,'durante'
               ,'sin'
               ,'coloca'
               ,'minutos']
    for w in words:
        st = w.split(',')
        l = []
        for s in st:
            x = s.split(' ')
            l = l + x
        l = [x.lower().strip() for x in l] #strip each word
        l = list(filter(lambda a: a not in delated, l)) # clean misleading words
        l = list(OrderedDict.fromkeys(l))  #Eliminate repeated words in a row
        Recetas.append(l)       #fd

    for r in Recetas:
        for j in range(len(r)): 
            if 'dole' in r[j]:
                r[j] = 'dole'
    return Recetas
    # eliminating next  part whe attempting the final model           
    # Recetas = Recetas[:]

Recetas = filterWords(data) 

Words = list(itertools.chain.from_iterable(Recetas))
Word_labels = list(OrderedDict.fromkeys(Words))
# W = pd.DataFrame(Words)


# W[0].value_counts()

# Word_labels
c = Counter(Words)
h = c.most_common(60)
h = array(h)
Word_labels = h[:,0]

In [ ]:
# Word_labels
# Recetas[0]
Recetas

In [ ]:
G = nx.Graph()
G.add_nodes_from(array(Word_labels))
Recetas_copy = Recetas.copy()
elist = []
nodeSize = zeros(len(Word_labels))

alpha = 1/len(Recetas)
# 'piña' in m
for i in range(len(Word_labels)):
    w = Word_labels[i]
    for j in arange(i+1 , len(Word_labels)):
        w2 = Word_labels[j]
#         print(w,w2)
        count = 0
        for R in Recetas:
            if (w in R) & (w2 in R):
                count = count +1
                nodeSize[i] = nodeSize[i] + 1
                nodeSize[j] = nodeSize[j] + 1
#                 print(count)
        if count >2:
            elist.append((w,w2,count*alpha))
G.add_weighted_edges_from(elist)
    
    
#     for R in Recetas_copy:
#         if (w in R):
#             for w2 in R:
#                 if (w2 in Word_labels):
#                     G.add_edge(w, w2)
#         found = False
#         while (found = False)
        
#             if 
eds = array(elist)
#Filter nodes coupled with dole    
G2 = nx.ego_graph(G, 'dole')

#The node size will be greater when the node is connected with more words
nodes = list(G2.nodes())
nodeSizes = zeros(len(nodes))
for i in range(len(nodes)):
    x = nodes[i]
    nodeSizes[i] = len([n for n in G2.neighbors(x)])
    
# nodeSizes
mini = min(eds[:,2])
maxi = max(eds[:,2])
edges,weights = zip(*nx.get_edge_attributes(G2,'weight').items())
# print(mini,maxi)
# eds[:,2]
# graphs = list(nx.connected_component_subgraphs(G))
# G2 = graphs[0]

In [ ]:
print(G2.number_of_nodes())
nx.draw_networkx( G2 , pos=nx.spring_layout(G2)
                 , node_size = nodeSize
                 , with_labels = True
                 , node_color= 'r'
                 , edgelist = edges
                 , width= 1.0
                 , edge_color = weights
                 , edge_cmap = cm.Blues
                 , edge_vmin = 0 
                 , edge_vmax = 0.6)
# print(nx.triangles(G,'piña'))

In [ ]:
print(random.randrange(10, 50, 5))

In [ ]:
y_true = np.random.randint(2, size=len(nodes))

In [ ]:
nx.draw_communities(G2, y_true, pos)

In [ ]:
# G = nx.complete_graph(5)
# K5 = nx.convert_node_labels_to_integers(G,first_label=2)
# G.add_edges_from(K5.edges())
c = list(k_clique_communities(G2, 4))
# print(list(c[0]))
# print(list(k_clique_communities(G2, 5)))
list(k_clique_communities(G2, 2))

In [ ]:
def graph_to_edge_matrix(G):
    """Convert a networkx graph into an edge matrix.
    See https://www.wikiwand.com/en/Incidence_matrix for a good explanation on edge matrices
   
    Parameters
    ----------
    G : networkx graph
    """
    G = nx.convert_node_labels_to_integers(G,first_label=0)
    # Initialize edge matrix with zeros
    edge_mat = np.zeros((len(G), len(G)), dtype=int)

    # Loop to set 0 or 1 (diagonal elements are set to 1)
    for node in G:
        for neighbor in G.neighbors(node):
            edge_mat[node][neighbor] = 1
#             print(node,neighbor)
        edge_mat[node][node] = 1

    return edge_mat
edge_mat = graph_to_edge_matrix(G2)

In [ ]:
algorithm = cluster.AffinityPropagation(damping=0.6)
algorithm.fit(edge_mat)
# Fit all models
for e,c in enumerate(algorithm.labels_):
     print(e,c)

In [ ]:
def draw_communities(G, membership):
    """Draws the nodes to a plot with assigned colors for each individual cluster
    Parameters
    ----------
    G : networkx graph
    membership : list
        A list where the position is the student and the value at the position is the student club membership.
        E.g. `print(membership[8]) --> 1` means that student #8 is a member of club 1.
    pos : positioning as a networkx spring layout
        E.g. nx.spring_layout(G)
    """ 
    fig, ax = subplots(figsize=(16,9))
    
    # Convert membership list to a dict where key=club, value=list of students in club
    club_dict = defaultdict(list)
    for student, club in enumerate(membership):
        club_dict[club].append(student)
    
    # Normalize number of clubs for choosing a color
    norm = colors.Normalize(vmin=0, vmax=len(club_dict.keys()))
    
    edges, weights = zip(*nx.get_edge_attributes(G,'weight').items())
    
    for club, members in club_dict.items():
        nx.draw_networkx( G , pos=nx.spring_layout(G)
                 , node_size = nodeSize
                 , with_labels = True
                 , nodelist=members
                 , node_color=cm.jet(norm(club))
                 , edgelist = edges
                 , width= 1.0
                 , edge_color = weights
                 , edge_cmap = cm.Blues
                 , edge_vmin = 0 
                 , edge_vmax = 0.6)

    # Draw edges (social connections) and show final plot
    title("Recepies")
#     nx.draw_networkx_edges(G, pos, alpha=0.5, ax=ax)

In [ ]:
draw_communities(G2,algorithm.labels_ )

In [ ]:
G=nx.path_graph(4)
pos=nx.spring_layout(G)
nx.draw_networkx( G, pos)

In [ ]:
nx.draw_networkx( G2)

In [ ]:
l = [1,2,4,1,3,4,6,7,8,1,4]
list(filter(lambda a: a not in [1 , 4, 8], l)) 

In [ ]:
dolead = G.adjacency()
# G.add_node(1)
# G.add_node('b')
G.subgraph(dolead['dole'])

In [ ]:
k_components = apxa.k_components(G)

In [ ]:
k_components[1]

In [ ]:
G = nx.petersen_graph()
k_components = apxa.k_components(G)

In [ ]:
k_components

In [ ]:
nx.draw_networkx(G)

In [ ]:
df['Ingredientes_ingrediente'].str.split(pat = "|",expand = True)

In [ ]:
display_all(df.describe(include='all').T)

In [ ]:
df['tags'] = df['Tags'].str.split(pat = "|")

In [ ]:
df[{'Title','tags'}]

In [ ]:
df.head()

In [ ]:
df['ingredientes_11_asterisco'].value_counts()

In [ ]:
s = {'Ingredientes_ingrediente', 'Ingredientes_cantidad' , 'Ingredientes_nombre_de_condicion'}
df[s]

In [ ]:
x = 'ingredientes_9_ingrediente'
ingredients = df[x].str.split(pat = "|",expand=True)
# ingredients[(ingredients[0].notnull()) & (ingredients[0] != ingredients[1]) & (ingredients[1].notnull()) ]
# ingredients = df.ingredientes_17_ingrediente.notnull()
# ingredients[df.ingredientes_17_ingrediente.notnull()]
# 1 in ingredients
num = x.split('_')
num = num[1]
print(num)
df['Ingrediente_'+ num] = ingredients[0] 

if 1 in ingredients:
                print('entered')
                condition = ((ingredients[0].notnull()) 
                           & (ingredients[0] != ingredients[1]) 
                           & (ingredients[1].notnull()))
                    
                
#                 df['Ingrediente_'+ num] = ingredients[0] +','+ ext[1]

df.loc[condition, 'Ingrediente_'+ num ] = (ingredients.loc[condition, 0]  + ','+ ingredients.loc[condition, 1] ) 
condition = ((ingredients[0].notnull()) 
                           & (ingredients[0] != ingredients[1]) 
                           & (ingredients[1].notnull()))

# df['Ingrediente_'+ num]
# ext
df.loc[condition,'Ingrediente_'+ num]

In [ ]:
display_all(df.describe(include='all').T)

In [ ]:
df['Nombre del Autor'].value_counts()

In [ ]:
# for x in df.columns:
#     if df[x].count() < 10:
#         del df[x]
df.head()

In [ ]:
df['Categories'].value_counts()

In [ ]:
df['ingredientes_0_cantidad'].value_counts()

In [ ]:
df['_ingredientes_0_cantidad'].value_counts()

In [ ]:
df['Content'].value_counts()